[2025-09-19 Fiddler](https://thefiddler.substack.com/p/can-your-team-self-organize)
====================

Fiddler
-------
With 4, we can enumerate every possibility.

1. 1234 - 4
2. 1243 - 3
3. 1324 - 3
4. 1342 - 3
5. 1423 - 3
6. 1432 - 2
7. 2134 - 3
8. 2143 - 2
9. 2314 - 3
10. 2341 - 3
11. 2413 - 2
12. 2431 - 2
13. 3124 - 3
14. 3142 - 2
15. 3214 - 2
16. 3241 - 2
17. 3412 - 2
18. 3421 - 2
19. 4123 - 3
20. 4132 - 2
21. 4213 - 2
22. 4231 - 2
23. 4312 - 2
24. 4321 - 1

And the average is $(4 + 9\cdot3 + 13\cdot2 + 1)/4! = 58/24 \approx 2.4167$.

Extra credit
------------
With 2,

1. 12 - 2
2. 21 - 1

for $3/2!$.

With 3,

1. 123 - 3
2. 132 - 2
3. 213 - 2
4. 231 - 2
5. 312 - 2
6. 321 - 1

for $12/3!$.

From 12 - 2, we can make a12, 1a2, 12a, where a is 0.5, 1.5, 2.5, so a12 can be 123 - 3, 213 - 2, 312 - 2,
1a2 can be 213 - 2, 123 - 3, 132 - 2, and 12a can be 231 - 2, 132 - 2, 123 - 3.  There must be some
way to combine the overlaps.  Abandoning this line of thought...

Looking up 1,3,12,58 in [OEIS](https://oeis.org), yields
[A003316, sum of lengths of longest increasing subsequences of all permutations of n elements](https://oeis.org/A003316),
which is pretty much what we're looking for.

And the answer is $15730705/10! = 3146141/725760 \approx 4.33$.

In [1]:
15730705/factorial(10), numerical_approx(15730705/factorial(10))

(3146141/725760, 4.33496059303351)

Revisiting
----------
When going from $n$ to $n+1$, one way to enumerate every permutation of $n+1$
is to insert a new lowest number into each of $n+1$ positions in each permutation
of $n$.  Another way is to add the new $i$-th largest to the beginning of
each permutation, where $i \in 1 \ldots n+1$, which is what I think I'll be
doing.

Given a permutation of $1\ldots n$, we can reduce that to $L_i$, where $i \in 1\ldots n$
and $L_i$ is the longest increasing subsequence starting with $i$ in the permutation.
Multiple permutations can be reduced to the same $L_i$, which I am counting on to rein
in the combinatorial explosion.  Finding $\max L_i$ for each permutation and summing them
should result the $n$-th term of A003316.

Given a $L_{n,i}$ for a permutation of $n$, we can generate $L_{n+1,i}$ for $n+1$
permutations of $n+1$.  Doing so for every permutation of $n$ will give $L_{n+1,i}$
for every permutation of $n+1$.

So, given a permutation of $n$ with $L_{n,i}$, and generating a permutation of $n+1$
by adding a new $k$-th smallest element at the beginning, gives

$$ L_{n+1,i} =
    \begin{cases}
        L_{n,i}, & i < k \\
        L_{n,i-1}, & i > k \\
        1 + \max L_{n,j}, & i = k, \text{where } j \in k \dots n, \\
        1, & i = k, \text{when } k = n+1 \\
    \end{cases}
$$

In [2]:
def n_plus_1(Ls):
    next_Ls = dict()
    for (L,count) in Ls.items():
        for k in range(len(L)+1):
            Lk = 1 + max([0]+list(L)[k:])
            next_L = tuple([L[i] if i < k else L[i-1] if i > k else Lk for i in range(len(L)+1)])
            if next_L in next_Ls:
                next_Ls[next_L] += count
            else:
                next_Ls[next_L] = count
    return next_Ls

def a3316(Ls):
    total = 0
    for (L,count) in Ls.items():
        total += count*max(L)
    return total

In [3]:
for i in [1..11]:
    L = {(1,):1} if i == 1 else n_plus_1(L)
    print(f"{a3316(L)}/{i}! ≈ {numerical_approx(a3316(L)/factorial(i))}")

1/1! ≈ 1.00000000000000
3/2! ≈ 1.50000000000000
12/3! ≈ 2.00000000000000
58/4! ≈ 2.41666666666667
335/5! ≈ 2.79166666666667
2261/6! ≈ 3.14027777777778
17465/7! ≈ 3.46527777777778
152020/8! ≈ 3.77033730158730
1473057/9! ≈ 4.05935019841270
15730705/10! ≈ 4.33496059303351
183571817/11! ≈ 4.59886105599647


The code becomes much too slow for $n > 10$.

### Numerical simulation
[Numerical simulations](20250919.go) agree

    $ go run 20250919.go
    1: 1.000000
    2: 1.499885
    3: 2.000064
    4: 2.416557
    5: 2.791281
    6: 3.140167
    7: 3.465389
    8: 3.769798
    9: 4.059213
    10: 4.335328
    11: 4.599312